In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astropy.io import ascii, fits
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.modeling.models import BlackBody
from scipy import stats
import astrotools.orbitparams as orb
import astrotools.generateExoplanetTable as genTab
import astrotools.interpolate_stellar_spectra as interpspec
import pysynphot as S
import pickle
import json
import starry as st
import copy
import os
import glob
from collections import OrderedDict
st.config.lazy = False
st.config.quiet = True
from jwst_backgrounds import jbt
import pandeia.engine
from pandeia.engine.calc_utils import build_default_calc
from pandeia.engine.instrument_factory import InstrumentFactory
from pandeia.engine.perform_calculation import perform_calculation

print("Pandeia engine versions:", pandeia.engine.pandeia_version())

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
/home/hannah/programs/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pandeia Engine version:  3.1
Pandeia RefData version:  3.0
Synphot Data:  /home/hannah/programs/grp/redcat/trds
Pandeia engine versions: None


In [23]:
# reference information
ref_star_name = 'TRAPPIST-1'
ref_targ_name = 'TRAPPIST-1 c'
ref_ecl_depth_err = 187 * 1e-6
ref_nobs = 1
ref_targ_dur = 42*u.min

In [24]:
ref_targ = genTab.generateExoSystem(hostname=ref_star_name)
ref_targ

Accessing NASA Exoplanet Archive
Table of transiting exoplanets retrieved
Writing table to ./NASAExoArchive_TransitingExoplanetSystem_TRAPPIST-1.dat


hostname,pl_name,sy_dist,sy_disterr1,sy_disterr2,rastr,decstr,sy_kmag,st_mass,st_masserr1,st_masserr2,st_rad,st_raderr1,st_raderr2,st_teff,st_tefferr1,st_tefferr2,st_tefferr12,st_tefferr22,st_logg,st_lum,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_rade,pl_radeerr1,pl_radeerr2,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbincl,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_ratror,pl_ratdor,tran_flag,rv_flag
,,pc,pc,pc,deg,deg,mag,Msun,Msun,Msun,Rsun,Rsun,Rsun,K,K,K,K,K,log(cm/s**2),log(Lsun),day,day,day,Rearth,Rearth,Rearth,Mearth,Mearth,Mearth,day,day,day,AU,AU,AU,deg,,,,,,,
object,object,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32
TRAPPIST-1,TRAPPIST-1 b,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457322.514193,3e-06,-3e-06,1.116,0.014,-0.012,1.374,0.069,-0.069,1.510826,6e-06,-6e-06,0.01154,0.0001,-0.0001,89.728,0.00622,0.00304,-0.00304,0.0859,20.843,1,0
TRAPPIST-1,TRAPPIST-1 c,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457282.811387,4e-06,-4e-06,1.097,0.014,-0.012,1.308,0.056,-0.056,2.421937,1.8e-05,-1.8e-05,0.0158,0.00013,-0.00013,89.778,0.00654,0.00188,-0.00188,0.0844,28.549,1,0
TRAPPIST-1,TRAPPIST-1 d,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457670.146301,1.8e-05,-1.8e-05,0.788,0.011,-0.01,0.388,0.012,-0.012,4.049219,2.6e-05,-2.6e-05,0.02227,0.00019,-0.00019,89.896,0.00837,0.00093,-0.00093,0.06063,40.216,1,0
TRAPPIST-1,TRAPPIST-1 e,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457660.367662,1.4e-05,-1.4e-05,0.92,0.013,-0.012,0.692,0.022,-0.022,6.101013,3.5e-05,-3.5e-05,0.02925,0.0025,-0.0025,89.793,0.0051,0.00058,-0.00058,0.07079,52.855,1,0
TRAPPIST-1,TRAPPIST-1 f,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457671.37373,1.6e-05,-1.6e-05,1.045,0.013,-0.012,1.039,0.031,-0.031,9.20754,3.2e-05,-3.2e-05,0.03849,0.00033,-0.00033,89.74,0.01007,0.00068,-0.00068,0.0804,69.543,1,0
TRAPPIST-1,TRAPPIST-1 g,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457665.362844,2.1e-05,-2.1e-05,1.129,0.015,-0.013,1.321,0.038,-0.038,12.352446,5.4e-05,-5.4e-05,0.04683,0.0004,-0.0004,89.742,0.00208,0.00058,-0.00058,0.08692,84.591,1,0
TRAPPIST-1,TRAPPIST-1 h,12.4298888,0.0187066,-0.0187066,23h06m30.33s,-05d02m36.46s,10.296,0.09,0.0,-0.0,0.12,0.0,-0.0,2566.0,26.0,-26.0,26.0,-26.0,5.24,-3.257,2457662.574149,9.1e-05,-9.1e-05,0.755,0.014,-0.014,0.326,0.02,-0.02,18.772866,0.000214,-0.000214,0.06189,0.00053,-0.00053,89.805,0.00567,0.00121,-0.00121,0.05809,111.817,1,0


In [9]:
def get_bkg(targ, ref_wave, make_new_bkg=False, savepath='../sample/background/'):
    """
    Code to retrieve sky background from the jwst backgrounds database based on system coordinates
    JWST backgrounds: https://jwst-docs.stsci.edu/jwst-general-support/jwst-background-model
    
    Inputs:
    targ          -- dictionary of target; must include RA and Dec of system as strings
    ref_wave      -- reference wavelength for jbt to return 
    make_new_bkg  -- default=True; otherwise you can load up the last background called, 
                                   use if only working with one system
                                   
    Returns:
    background    -- list of two lists containing wavelength (um) and background counts (mJy/sr)
    """
    
    print('Computing background')

    sys_coords = targ['rastr']+' '+targ['decstr']
    sys_name   = targ['hostname'].replace(" ", "")

    def bkg():
    
        # make astropy coordinates object
        c = SkyCoord(sys_coords, unit=(u.hourangle, u.deg))

        # use jwst backgrounds to compute background at this point
        bg = jbt.background(c.ra.deg, c.dec.deg, ref_wave)

        # background is computed for many days; choose one
        ndays = bg.bkg_data['calendar'].size
        assert ndays > 0  # make sure object is visible at some point in the year; if not check coords
        middleday = bg.bkg_data['calendar'][int(ndays / 2)] # picking the middle visible date; somewhat arbitrary!
        middleday_indx = np.argwhere(bg.bkg_data['calendar'] == middleday)[0][0]

        tot_bkg = bg.bkg_data['total_bg'][middleday_indx]
        wav_bkg = bg.bkg_data['wave_array']

        # background is [wavelength, total_background] in [micron, mJy/sr]
        background = [list(np.array(wav_bkg)), list(np.array(tot_bkg))]

        ascii.write(background, savepath+f"{sys_name}_background.txt", overwrite=True)
        
        print('Returning background')
        return background
    
    if make_new_bkg: background = bkg()

    else: 
        try: 
            print('Using existing background')
            background = ascii.read(savepath+f"{sys_name}_background.txt")
            background = [list(background['col0']), list(background['col1'])]
        except:
            background = bkg()
    
    return background

In [10]:
def make_miri_dict(filter, subarray, targ, pull_default_dict=True, scene_spec='phoenix', key='m5v', input_spec=[]):
    """
    Code to make the initial miri dictionally for imaging_ts
    
    Inputs:
    filter            -- which photometric filter to use (e.g., f1500w)
    subarray          -- which subarray readout ot use (e.g., sub256)
    targ              -- 
    sys_coords        -- string of the coordinates of the system in RA Dec; e.g. "23h06m30.33s -05d02m36.46s";
                         to be passed to get_bkg function
    pull_default_dict -- default=True; can re-use a saved one but this doesn't save much time.
    scene_spec        -- default='phoenix' this default requires the "key" argument
                         can also be 'input' which then requires the "input_spec" argument to not be empty
    key               -- goes with scene_spec;  (e.g., 'm0v, m5v')
    input_spec        -- list of 2 arrays, or ndarray; Wavelength (μm) and Flux (mJy) arrays of the SED to use;
                         In an ndarray, wavelength is the 0th index, and flux the 1st index.
    """

    print('Creating MIRI dictionary')

    # grab default imaging ts dictionary (I think this only works online?)
    if pull_default_dict:
        miri_imaging_ts = build_default_calc('jwst', 'miri', 'imaging_ts')

        # Serializing json
        json_object = json.dumps(miri_imaging_ts, indent=4)

        # Writing to sample.json
        with open("miri_imaging_ts.json", "w") as outfile:
            outfile.write(json_object)

    else: 
        with open("miri_imaging_ts.json", "r") as f:
            miri_imaging_ts = json.load(f)
            
    if   filter == 'f1500w': ref_wave = 15 * u.micron
    elif filter == 'f1800w': ref_wave = 18 * u.micron
            
    # update with basic parameters
    miri_imaging_ts['configuration']['instrument']['filter'] = filter
    miri_imaging_ts['configuration']['detector']['subarray'] = subarray
       
    miri_imaging_ts['configuration']['detector']['ngroup']   = 2    
    miri_imaging_ts['configuration']['detector']['nint'] = 1 
    miri_imaging_ts['configuration']['detector']['nexp'] = 1
    miri_imaging_ts['configuration']['detector']['readout_pattern'] = 'fastr1'
    try: miri_imaging_ts['configuration'].pop('max_filter_leak')
    except(KeyError): pass

    miri_imaging_ts['scene'][0]['spectrum']['normalization'] = {}
    miri_imaging_ts['scene'][0]['spectrum']['normalization']['type']          = 'photsys'
    miri_imaging_ts['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'vegamag'
    miri_imaging_ts['scene'][0]['spectrum']['normalization']['bandpass']      = '2mass,ks'
    miri_imaging_ts['scene'][0]['spectrum']['normalization']['norm_flux']     = targ['sy_kmag']           # change this for different stars

    if scene_spec == 'phoenix': 
        miri_imaging_ts['scene'][0]['spectrum']['sed']['key']          = key
        miri_imaging_ts['scene'][0]['spectrum']['sed']['sed_type']     = 'phoenix'
        try: miri_imaging_ts['scene'][0]['spectrum']['sed'].pop('unit')
        except(KeyError): pass
    elif scene_spec == 'input':
        miri_imaging_ts['scene'][0]['spectrum']['sed']['sed_type']     = 'input'
        miri_imaging_ts['scene'][0]['spectrum']['sed']['spectrum']     = input_spec
        try: miri_imaging_ts['scene'][0]['spectrum']['sed'].pop('unit')
        except(KeyError): pass
        try: miri_imaging_ts['scene'][0]['spectrum']['sed'].pop('key')
        except(KeyError): pass

    miri_imaging_ts['background'] = get_bkg(targ, ref_wave)
    miri_imaging_ts['background_level'] = 'high'

    miri_imaging_ts['strategy']['aperture_size']  = 0.55            # values from Greene+ 2023 for T1-b
    miri_imaging_ts['strategy']['sky_annulus']    = [1.32, 2.8]     # assuming MIRI plate scale of 0.11"/pix

    print('Returning MIRI dictionary')
    return miri_imaging_ts

In [11]:
sample = ascii.read('./sample_final.csv')
sample

hostname,pl_name,sy_dist,rastr,decstr,sy_kmag,st_mass,st_rad,st_raderr1,st_raderr2,st_teff,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_rade,pl_radeerr1,pl_radeerr2,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbsmax,pl_orbincl,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,tran_flag,rv_flag,pl_rp_rs,pl_g,pl_teq_a0,pl_s,pl_rho,filter,subarray,nobs,ngroup,frac_fullwell
str9,str11,float64,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,str6,str6,int64,int64,float64
GJ 3473,GJ 3473 b,27.3644,08h02m22.47s,+03d20m13.79s,8.829001,0.36,0.36,0.01,-0.01,3347.0,2458492.20412163,0.00037015,-0.00037015,1.264,0.05,-0.049,1.86,0.3,-0.3,1.198003975976397,5.76582216e-07,-5.76582216e-07,0.01589,87.95,0.0,--,--,1,1,0.0321894750291473,11.407064002554433,768.2070915703122,57.83231621613784,5.06105605260494,f1500w,sub256,4,39,0.65
GJ 357,GJ 357 b,9.44181,09h36m01.79s,-21d39m54.23s,6.475,0.34,0.34,0.02,-0.02,3505.0,2458517.998679131,0.0003765,-0.0003765,1.217,0.084,-0.083,1.84,0.31,-0.31,3.9306063296116895,1.16151077e-06,-1.16151077e-06,0.035,89.12,--,--,--,1,1,0.03281564780289002,12.17283537042799,526.7769415618036,12.786873373040743,5.609388414817885,f1500w,sub64,1,22,0.85
HD 260655,HD 260655 b,10.0055,06h37m09.97s,+17d33m58.57s,5.862,0.44,0.439,0.003,-0.003,3803.0,2459497.9102,0.0003,-0.0003,1.24,0.023,-0.023,2.14,0.34,-0.34,2.76953,3e-05,-3e-05,0.02933,87.35,0.039,0.043,-0.023,1,1,0.02589563026767422,13.63720864077306,709.4729664292033,42.072735108287475,6.167627593216154,f1500w,sub64,2,16,0.95
L 98-59,L 98-59 c,10.6194,08h18m07.89s,-68d18m52.08s,7.101,0.27,0.3,0.03,-0.02,3415.0,2458367.27375,0.00013,-0.00022,1.34,0.07,-0.07,2.42,0.35,-0.34,3.6906777,1.6e-06,-2.6e-06,0.0304,88.11,0.103,0.045,-0.058,1,1,0.04094990177758612,13.20568249247054,517.3063880405923,11.891830329088151,5.5267573282965365,f1500w,sub128,1,23,0.7
LHS 1140,LHS 1140 c,14.9861,00h44m59.67s,-15d16m26.79s,8.821,0.19,0.21,0.0035,-0.0035,2988.0,2458389.2942323,0.00062395,-0.00062395,1.169,0.037,-0.038,1.76,0.17,-0.16,3.777939728,5.83523921e-06,-5.83523921e-06,0.02734,89.913,0.274,--,--,1,1,0.05103457908102151,12.61940060872384,399.3229614135901,4.2223523461028085,6.053946228618852,f1500w,sub256,3,36,0.65
LHS 1478,LHS 1478 b,18.2276,02h57m21.33s,+76d33m05.06s,8.767,0.24,0.25,0.01,-0.01,3381.0,2458786.75425,0.00042,-0.00042,1.242,0.051,-0.049,2.33,0.2,-0.2,1.9495378,4e-06,-4.1e-06,0.01848,87.452,--,--,--,1,1,0.045546069857697276,14.800207739484296,599.6495187649439,21.470754020600786,6.682832228969028,f1500w,sub256,2,39,0.65
LTT 3780,LTT 3780 b,21.9814,10h18m34.78s,-11d43m04.08s,8.204001,0.4,0.37,0.01,-0.01,3331.0,2458543.91281,0.00048,-0.00052,1.32,0.06,-0.06,2.47,0.24,-0.24,0.768377,1.4e-06,-1.4e-06,0.01203,85.9,0.064,0.075,-0.046,1,1,0.032707061524655315,13.89006163347313,890.7913247681156,104.55895980434461,5.901257760073781,f1500w,sub256,2,22,0.65
TOI-1468,TOI-1468 b,24.7399,01h06m36.93s,+19d13m29.71s,8.497,0.34,0.34,0.01,-0.01,3496.0,2458765.68079,0.0007,-0.00069,1.28,0.038,-0.039,3.21,0.24,-0.24,1.8805136,2.4e-06,-2.6e-06,0.02102086,88.47,--,--,--,1,1,0.03451440360534036,19.197301030931996,677.9832124158797,35.08593869199558,8.410940260139574,f1500w,sub256,3,31,0.65
TOI-270,TOI-270 b,22.453,04h33m39.86s,-51d57m26.62s,8.251,0.386,0.38,0.008,-0.008,3506.0,2458387.09505,0.00074,-0.00074,1.206,0.039,-0.039,1.58,0.26,-0.26,3.3601538,4.8e-06,-4.8e-06,0.03197,89.39,0.034,0.025,-0.025,1,1,0.029095982841969084,10.64431095709798,582.8619296384281,19.165497515761714,4.949765091265365,f1500w,sub256,4,25,0.65


In [65]:
# make the input spectra for each star; will require going into literature
# save spectra with the columns: wavelength (μm) and flux (mJy) 
stellar_spec_directory = '../sample/SPHINX_stellar_spectra/'
saved_spec = os.listdir(stellar_spec_directory)

def save_input_spec(spec, starname):
    wave = spec['Wavelength(A)'].to(u.micron)
    flux = (spec['Flux(erg/cm2/s/A)']*wave/(const.c/wave)).to(u.mJy)
    ascii.write([wave, flux], f'{stellar_spec_directory}{starname}_SPHINX_spec.dat', names=['#wave(um)', 'fluxden(mJy)'], overwrite=True)
    
if 'GJ3473_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3347, logg=4.81, logz=0.0), starname='GJ3473')
if 'GJ357_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3505, logg=4.94, logz=0.0), starname='GJ357')
if 'HD260655_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3803, logg=5.20, logz=-0.43), starname='HD260655')
if 'L98-59_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3412, logg=4.94, logz=-0.5), starname='L98-59')
if 'LHS1140_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=2988, logg=5.037, logz=-0.262), starname='LHS1140')
if 'LHS1478_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3381, logg=4.87, logz=-0.13), starname='LHS1478')
if 'LTT3780_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3360, logg=4.81, logz=0.090), starname='LTT3780')
if 'TOI-1468_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3496, logg=5.00, logz=-0.040), starname='TOI-1468')
if 'TOI-270_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=3506, logg=4.872, logz=-0.20), starname='TOI-270')
if 'TRAPPIST-1_SPHINX_spec.dat' in saved_spec: pass
else: save_input_spec(interpspec.interp_spec(model='SPHINX', Teff=2566, logg=5.2396, logz=0.0535), starname='TRAPPIST-1')

In [82]:
default_filter    = 'f1500w'
default_subarray  = 'sub256'

In [105]:
input_spec_path = f'{stellar_spec_directory}{ref_star_name}_SPHINX_spec.dat'
input_spec = ascii.read(input_spec_path)
input_spec = np.array([input_spec['wave(um)'], input_spec['fluxden(mJy)']])

ref_miri_imaging_ts = make_miri_dict(default_filter, default_subarray, ref_targ[1], scene_spec='input', input_spec=input_spec)
ref_report = perform_calculation(ref_miri_imaging_ts)
ref_flux = ref_report['1d']['extracted_flux'][1]

i = InstrumentFactory(ref_miri_imaging_ts['configuration'])
#miri_imaging_ts['configuration']
i.read_detector_pars()['rn']

ref_report['1d']

Creating MIRI dictionary
Computing background
Using existing background
Returning MIRI dictionary


{'wave_pix': array([14.81601225]),
 'wave_calc': array([12.5       , 12.53235333, 12.58248274, 12.63281267, 12.68334392,
        12.7       , 12.7340773 , 12.78501361, 12.83615366, 12.88749828,
        12.9       , 12.93904827, 12.99080446, 13.04276768, 13.09493875,
        13.1       , 13.14731851, 13.19990778, 13.25270741, 13.3       ,
        13.30571824, 13.35894111, 13.41237688, 13.46602639, 13.5       ,
        13.51989049, 13.57397005, 13.62826593, 13.682779  , 13.7       ,
        13.73751011, 13.79246015, 13.84763   , 13.9       , 13.90302052,
        13.9586326 , 14.01446713, 14.070525  , 14.1       , 14.1268071 ,
        14.18331432, 14.24004758, 14.29700777, 14.3       , 14.3541958 ,
        14.41161259, 14.46925904, 14.5       , 14.52713607, 14.58524462,
        14.6435856 , 14.7       , 14.70215994, 14.76096858, 14.82001245,
        14.8792925 , 14.9       , 14.93880967, 14.99856491, 15.05855917,
        15.1       , 15.11879341, 15.17926858, 15.23998565, 15.3       ,
   

In [84]:
targ = sample[0]
hostname = targ['hostname']
input_spec_path = f'{stellar_spec_directory}{hostname.replace(" ", "")}_SPHINX_spec.dat'
input_spec = ascii.read(input_spec_path)
input_spec = np.array([input_spec['wave(um)'], input_spec['fluxden(mJy)']])
nobs = 4
targ_dur = orb.Tdur(P=targ['pl_orbper']*u.day, 
                    Rp_Rs=((targ['pl_rade']*u.R_earth)/(targ['st_rad']*u.R_sun)).decompose().value,
                    a_Rs = ((targ['pl_orbsmax']*u.AU)/(targ['st_rad']*u.R_sun)).decompose().value,
                    i = targ['pl_orbincl']
                   ) # event duration
#pull_default_dict = True # needs to be true if you want to pull a new default dictionary from pandeia
#make_new_bkg      = True # should be true if working with different stars; otherwise set to False for speed
targ_miri_imaging_ts = make_miri_dict(default_filter, default_subarray, targ, scene_spec='input', input_spec=input_spec)
targ_report = perform_calculation(targ_miri_imaging_ts)
targ_flux = targ_report['1d']['extracted_flux'][1]
#targ_report

Creating MIRI dictionary
Computing background
Using existing background
Returning MIRI dictionary


In [85]:
scaled_err = (ref_ecl_depth_err * (np.sqrt(ref_nobs)/np.sqrt(nobs)) * (np.sqrt(ref_flux)/np.sqrt(targ_flux)) * (np.sqrt(ref_targ_dur)/np.sqrt(targ_dur))).decompose()
scaled_err * 1e6 , scaled_err * 1e6 / np.sqrt(2)

(<Quantity [44.99643908]>, <Quantity [31.8172872]>)